In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, Input

In [2]:
# Convert cell to numeric values
def convert_to_numeric(cell):
    return ord(cell[1]) - ord('A') + 1, int(cell[2])

# Read data from CSV file and preprocess it
def read_data_from_csv(file_path):
    data = pd.read_csv(file_path)

    # Preprocessing
    data[['ilk_konum_sayi']] = data['ilk_kare'].apply(lambda x: pd.Series(ord(x[1]) - ord('A')+1))
    data[['ilk_konum_sayi2']] = data['ilk_kare'].apply(lambda x: pd.Series(int(x[2])))
    data[['son_konum_sayi', 'son_konum_sayi2']] = data['son_konum'].apply(lambda x: pd.Series(convert_to_numeric(x)))

    # Drop the original columns
    data.drop(['ilk_kare', 'son_konum'], axis=1, inplace=True)

    return data

# Read data from CSV file
file_path = "sample_data/veri.csv"  # CSV dosya yolu
data = read_data_from_csv(file_path)

# Generate X and y
X = data[['durum', 'ilk_konum_sayi', 'ilk_konum_sayi2']]
y_position = data[['son_konum_sayi', 'son_konum_sayi2','son_durum']]

# One-hot encoding
X = pd.get_dummies(X, columns=['durum'], prefix='durum')
y_position = pd.get_dummies(y_position, columns = ["son_durum"], prefix="son_durum")
print(X.head(10))
print(X.shape[1])

print(y_position.head(10))

# Create the model
model = models.Sequential([
    Input(shape=(5,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='relu'), 
    layers.Dense(4, activation='relu')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error',
              loss_weights=0.5, metrics=['accuracy'])

# Seperate train and test data
X_train, X_test, y_position_train, y_position_test = train_test_split(
    X, y_position.values, test_size=0.2, random_state=42
)

model.summary()

# Shape validation
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],))
y_position = np.reshape(y_position,(y_position.shape[0],y_position.shape[1],))

print(y_position.shape)
# Training
model.fit(X_train,  y_position_train, epochs=50, validation_split=0.1)

# Calculate losses
losses = model.evaluate(X_test,  y_position_test)
print(f"Total loss and Accuracy: {losses}")

# Example prediction test
test_input = pd.DataFrame({'durum_yazi': [0], 'durum_tura': [1], 'durum_dik': [0], 'ilk_konum_sayi': [3], 'ilk_konum_sayi2' : [5]})
predicted_position = model.predict(test_input)

print(f"Predicted position and orientation: {predicted_position}")

   ilk_konum_sayi  ilk_konum_sayi2  durum_dik  durum_tura  durum_yazi
0               5                5          0           0           1
1               6                6          0           0           1
2               5                5          0           0           1
3               6                6          0           0           1
4               6                6          0           0           1
5               6                7          0           0           1
6               6                6          0           0           1
7               5                5          0           0           1
8               5                5          0           0           1
9               6                5          0           0           1
5
   son_konum_sayi  son_konum_sayi2  son_durum_ tura  son_durum_ yazi
0               3                3                0                1
1               6                7                0                1
2               7    

In [7]:
# Test with a real example
test_input = pd.DataFrame({'durum_yazi': [1], 'durum_tura': [0], 'durum_dik': [0], 'ilk_konum_sayi': [5], 'ilk_konum_sayi2' : [5]})
predicted_position = model.predict(test_input)
print(predicted_position)

1/1 [==============================] - 0s 28ms/step
[[2.4309165 2.6128309 1.        0.       ]]
